In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import json
from sklearn.linear_model import RidgeCV, Ridge, LinearRegression
from pathlib import Path
from scipy.stats import pearsonr
import pandas as pd
import nibabel as nib
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import random
import optuna 
import pickle
import gc
import sys

sys.path.append('..')
from src.train_utils import load_fmri, load_stimulus_features, align_features_and_fmri_samples, plot_correlations, train_model, training
from src.optimization import objective

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(777)

In [3]:
root_data_dir = 'data'
excluded_samples_start = 5  
excluded_samples_end = 5  
hrf_delay = 3
stimulus_window = 1
movies_train = ["friends-s01", "friends-s02", "friends-s03", "friends-s04", "friends-s05"] 
movies_test = ["friends-s06"] 

In [4]:
fmri_1 = load_fmri(root_data_dir, 1)
fmri_2 = load_fmri(root_data_dir, 2)
fmri_3 = load_fmri(root_data_dir, 3)
fmri_5 = load_fmri(root_data_dir, 5)

In [5]:
features_train, features_test = load_stimulus_features(method='fasttext', layer='embeddings')

0it [00:00, ?it/s]

5it [00:18,  3.68s/it]


(114294, 600)
(23387, 600)
3172800


In [6]:
x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

Before NaN removal: (111343, 600)
After NaN removal: (83858, 600)
Removed 27485 NaN samples (24.7%)
Before NaN removal: (111343, 600)
After NaN removal: (83858, 600)
Removed 27485 NaN samples (24.7%)
Before NaN removal: (110900, 600)
After NaN removal: (83531, 600)
Removed 27369 NaN samples (24.7%)
Before NaN removal: (109998, 600)
After NaN removal: (82874, 600)
Removed 27124 NaN samples (24.7%)
Before NaN removal: (22774, 600)
After NaN removal: (17832, 600)
Removed 4942 NaN samples (21.7%)
Before NaN removal: (22774, 600)
After NaN removal: (17832, 600)
Removed 4942 NaN samples (21.7%)
Before NaN removal: (22348, 600)
After NaN removal: (17473, 600)
Removed 4875 NaN samples (21.8%)
Before NaN removal: (22774, 600)
After NaN removal: (17832, 600)
Removed 4942 NaN samples (21.7%)


In [7]:
for k in y_train_1.keys():
    print(f"Train ROI {k}: {y_train_1[k].shape}, Test ROI {k}: {y_test_1[k].shape}")

Train ROI LH_IFGorb: (83858, 6), Test ROI LH_IFGorb: (17832, 6)
Train ROI LH_IFG: (83858, 11), Test ROI LH_IFG: (17832, 11)
Train ROI LH_MFG: (83858, 7), Test ROI LH_MFG: (17832, 7)
Train ROI LH_AntTemp: (83858, 17), Test ROI LH_AntTemp: (17832, 17)
Train ROI LH_PostTemp: (83858, 40), Test ROI LH_PostTemp: (17832, 40)


In [8]:
class OneHiddenMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.model(x)
    
def mlp(input_dim, hidden_dim, output_dim, dropout_rate):
    return OneHiddenMLP(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        dropout_rate=dropout_rate
    )

In [9]:
def train_and_save_best_models(x_train_list, y_train_list, x_test_list, y_test_list,
                                study, layer, fedorenko_zone, save_dir='results',
                                num_epochs=20):
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Extract best hyperparameters directly from study
    lr = study.best_params['lr']
    decay = study.best_params['decay']
    hidden_dim = study.best_params['hidden_dim']
    batch_size = study.best_params['batch_size']
    dropout_rate = study.best_params['dropout_rate']
    
    
    subjects = [1, 2, 3, 5]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models = []
    predictions_list = []
    
    for i, subject_id in enumerate(subjects):
        print(f"Training Subject {subject_id}")
        
        features_train = x_train_list[i]
        fmri_train = y_train_list[i]
        features_val = x_test_list[i]
        fmri_val = y_test_list[i]
        
        # Create model with best hyperparameters
        input_dim = features_train.shape[1]
        output_dim = fmri_train.shape[1]
        
        model = mlp(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout_rate=dropout_rate
        )
        
        # Train model
        trained_model = train_model(
            X_train=features_train,
            Y_train=fmri_train,
            X_val=features_val,
            Y_val=fmri_val,
            model_class=lambda: mlp(input_dim, hidden_dim, output_dim, dropout_rate),
            decay=decay,
            lr=lr,
            batch_size=batch_size,
            num_epochs=num_epochs,
            plot_curve=False
        )
        
        # Save model 
        model_save_path = save_dir / f'subject_{subject_id}_layer{layer}_zone{fedorenko_zone}_model.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'hyperparameters': {
                'input_dim': input_dim,
                'hidden_dim': hidden_dim,
                'output_dim': output_dim,
                'dropout_rate': dropout_rate,
                'lr': lr,
                'decay': decay,
                'batch_size': batch_size
            },
            'subject_id': subject_id
        }, model_save_path)
        print(f"Model saved: {model_save_path}")
        
        # Save complete model (for easier loading)
        model_full_path = save_dir / f'subject_{subject_id}_layer{layer}_zone{fedorenko_zone}_model_full.pkl'
        with open(model_full_path, 'wb') as f:
            pickle.dump(trained_model, f)
        print(f"Full model saved: {model_full_path}")
        
        # Generate predictions
        trained_model = trained_model.to(device)
        trained_model.eval()
        
        with torch.no_grad():
            X_test = torch.tensor(features_val, dtype=torch.float32).to(device)
            y_pred = trained_model(X_test)
            y_pred = y_pred.cpu().numpy()
        
        predictions_list.append(y_pred)
        
        # Keep model on CPU for Integrated Gradients
        trained_model = trained_model.cpu()
        models.append(trained_model)
        
        print(f"Subject {subject_id} - Predictions shape: {y_pred.shape}")
        
        # Cleanup
        del X_test
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    return predictions_list

# LH_IFGorb

In [10]:
subjects = [1, 2, 3, 5]
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

In [18]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_ft_IFGorb',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-12-01 13:43:48,323] A new study created in RDB with name: fmri_optimization_ft_IFGorb


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=5.89e-05, decay=4.49e-03, hidden=512, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.46s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:17<00:00,  1.12it/s]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.05it/s]


 Final score (mean r): 0.0451
[I 2025-12-01 15:56:46,400] Trial 49 finished with value: 0.045113496482372284 and parameters: {'lr': 7.056518944297948e-05, 'decay': 0.0055713414962050775, 'hidden_dim': 128, 'batch_size': 1024, 'dropout_rate': 0.5061538156493264}. Best is trial 45 with value: 0.04872763901948929.


In [21]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_IFGorb', save_dir='results/fasttext')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]


Model saved: results\fasttext\subject_1_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\fasttext\subject_1_layer1_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17832, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it]


Model saved: results\fasttext\subject_2_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\fasttext\subject_2_layer1_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17832, 6)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.48s/it]


Model saved: results\fasttext\subject_3_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\fasttext\subject_3_layer1_zoneLH_IFGorb_model_full.pkl
Subject 3 - Predictions shape: (17473, 6)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [01:39<00:00,  4.99s/it]


Model saved: results\fasttext\subject_5_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\fasttext\subject_5_layer1_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17832, 6)


In [22]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/fasttext'
)

Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0563, mean MSE = 0.3437
  Subject 2: mean r = 0.0529, mean MSE = 0.3069
  Subject 3: mean r = 0.0425, mean MSE = 0.3643
  Subject 4: mean r = 0.0324, mean MSE = 0.4013
Mean across subjects: r = 0.0460, MSE = 0.3540
Saved: results\fasttext\voxel_correlations_LH_IFGorb_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\fasttext\correlation_map_LH_IFGorb_layer1.png


# LH_IFG

In [23]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

In [24]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_ft_IFG',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-12-01 16:34:57,618] A new study created in RDB with name: fmri_optimization_ft_IFG


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=2.15e-04, decay=4.47e-03, hidden=512, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [01:12<00:00,  3.64s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [01:05<00:00,  3.28s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [01:04<00:00,  3.21s/it]


 Final score (mean r): 0.0836
[I 2025-12-01 18:59:45,293] Trial 49 finished with value: 0.08360527455806732 and parameters: {'lr': 3.805008639398303e-05, 'decay': 0.001981057118848468, 'hidden_dim': 1024, 'batch_size': 512, 'dropout_rate': 0.5141914070199675}. Best is trial 41 with value: 0.08533836901187897.


In [27]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_IFG', save_dir='results/fasttext')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


Model saved: results\fasttext\subject_1_layer1_zoneLH_IFG_model.pth
Full model saved: results\fasttext\subject_1_layer1_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17832, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:17<00:00,  1.16it/s]


Model saved: results\fasttext\subject_2_layer1_zoneLH_IFG_model.pth
Full model saved: results\fasttext\subject_2_layer1_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17832, 11)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.05it/s]


Model saved: results\fasttext\subject_3_layer1_zoneLH_IFG_model.pth
Full model saved: results\fasttext\subject_3_layer1_zoneLH_IFG_model_full.pkl
Subject 3 - Predictions shape: (17473, 11)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:20<00:00,  1.03s/it]


Model saved: results\fasttext\subject_5_layer1_zoneLH_IFG_model.pth
Full model saved: results\fasttext\subject_5_layer1_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17832, 11)


In [28]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_IFG',
    output_dir='./results/fasttext'
)

Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.0847, mean MSE = 0.3382
  Subject 2: mean r = 0.0994, mean MSE = 0.3046
  Subject 3: mean r = 0.0878, mean MSE = 0.3945
  Subject 4: mean r = 0.0670, mean MSE = 0.3469
Mean across subjects: r = 0.0847, MSE = 0.3461
Saved: results\fasttext\voxel_correlations_LH_IFG_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\fasttext\correlation_map_LH_IFG_layer1.png


## LH_MFG

In [29]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

In [30]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_ft_MFG',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-12-02 11:16:27,134] A new study created in RDB with name: fmri_optimization_ft_MFG


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=5.39e-04, decay=1.39e-03, hidden=32, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.01it/s]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.73s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.53s/it]


 Final score (mean r): 0.0666
[I 2025-12-02 15:08:11,010] Trial 49 finished with value: 0.06658945232629776 and parameters: {'lr': 0.0002060258488614142, 'decay': 0.0041088226453752374, 'hidden_dim': 256, 'batch_size': 1024, 'dropout_rate': 0.5977196837895766}. Best is trial 27 with value: 0.06662224978208542.


In [31]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_MFG', save_dir='results/fasttext')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]


Model saved: results\fasttext\subject_1_layer1_zoneLH_MFG_model.pth
Full model saved: results\fasttext\subject_1_layer1_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17832, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [01:17<00:00,  3.89s/it]


Model saved: results\fasttext\subject_2_layer1_zoneLH_MFG_model.pth
Full model saved: results\fasttext\subject_2_layer1_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17832, 7)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:52<00:00,  2.61s/it]


Model saved: results\fasttext\subject_3_layer1_zoneLH_MFG_model.pth
Full model saved: results\fasttext\subject_3_layer1_zoneLH_MFG_model_full.pkl
Subject 3 - Predictions shape: (17473, 7)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.76s/it]


Model saved: results\fasttext\subject_5_layer1_zoneLH_MFG_model.pth
Full model saved: results\fasttext\subject_5_layer1_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17832, 7)


In [32]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_MFG',
    output_dir='./results/fasttext'
)

Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.0704, mean MSE = 0.3140
  Subject 2: mean r = 0.0648, mean MSE = 0.2666
  Subject 3: mean r = 0.0795, mean MSE = 0.3226
  Subject 4: mean r = 0.0517, mean MSE = 0.3454
Mean across subjects: r = 0.0666, MSE = 0.3122
Saved: results\fasttext\voxel_correlations_LH_MFG_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\fasttext\correlation_map_LH_MFG_layer1.png


# LH_AntTemp

In [33]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

In [34]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_ft_AntTemp',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-12-02 16:22:46,466] A new study created in RDB with name: fmri_optimization_ft_AntTemp


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=2.07e-05, decay=4.35e-04, hidden=32, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:22<00:00,  1.10s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:27<00:00,  1.38s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:27<00:00,  1.37s/it]


 Final score (mean r): 0.1446
[I 2025-12-02 19:12:10,963] Trial 49 finished with value: 0.14458619058132172 and parameters: {'lr': 0.0003695948094300052, 'decay': 0.00012186779717340954, 'hidden_dim': 512, 'batch_size': 2048, 'dropout_rate': 0.4695673352104205}. Best is trial 18 with value: 0.14723610877990723.


In [35]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_AntTemp', save_dir='results/fasttext')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.70s/it]


Model saved: results\fasttext\subject_1_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\fasttext\subject_1_layer1_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17832, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.97s/it]


Model saved: results\fasttext\subject_2_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\fasttext\subject_2_layer1_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17832, 17)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]


Model saved: results\fasttext\subject_3_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\fasttext\subject_3_layer1_zoneLH_AntTemp_model_full.pkl
Subject 3 - Predictions shape: (17473, 17)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.68s/it]


Model saved: results\fasttext\subject_5_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\fasttext\subject_5_layer1_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17832, 17)


In [36]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/fasttext'
)

Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1345, mean MSE = 0.3003
  Subject 2: mean r = 0.1361, mean MSE = 0.2981
  Subject 3: mean r = 0.1730, mean MSE = 0.3566
  Subject 4: mean r = 0.1434, mean MSE = 0.3025
Mean across subjects: r = 0.1467, MSE = 0.3144
Saved: results\fasttext\voxel_correlations_LH_AntTemp_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\fasttext\correlation_map_LH_AntTemp_layer1.png


# LH_PostTemp

In [11]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

In [12]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_ft_PostTemp',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-12-03 21:33:27,650] Using an existing study with name 'fmri_optimization_ft_PostTemp' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 9
  lr=2.62e-05, decay=1.45e-03, hidden=512, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [01:42<00:00,  5.11s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]


 Final score (mean r): 0.1519
[I 2025-12-04 08:50:51,649] Trial 58 finished with value: 0.15192583203315735 and parameters: {'lr': 0.00014415170768550218, 'decay': 0.00021548781270079866, 'hidden_dim': 512, 'batch_size': 1024, 'dropout_rate': 0.5386357486272806}. Best is trial 34 with value: 0.15256989002227783.


In [13]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_PostTemp', save_dir='results/fasttext')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]


Model saved: results\fasttext\subject_1_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\fasttext\subject_1_layer1_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17832, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.47s/it]


Model saved: results\fasttext\subject_2_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\fasttext\subject_2_layer1_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17832, 40)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it]


Model saved: results\fasttext\subject_3_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\fasttext\subject_3_layer1_zoneLH_PostTemp_model_full.pkl
Subject 3 - Predictions shape: (17473, 40)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.45s/it]


Model saved: results\fasttext\subject_5_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\fasttext\subject_5_layer1_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17832, 40)


In [14]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/fasttext'
)

Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1342, mean MSE = 0.3839
  Subject 2: mean r = 0.1425, mean MSE = 0.3569
  Subject 3: mean r = 0.1760, mean MSE = 0.4094
  Subject 4: mean r = 0.1538, mean MSE = 0.3683
Mean across subjects: r = 0.1516, MSE = 0.3796
Saved: results\fasttext\voxel_correlations_LH_PostTemp_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\fasttext\correlation_map_LH_PostTemp_layer1.png
